In [1]:
# import pyspark 
# from pyspark.sql import SparkSession 
# from pyspark.sql import Row 



# spark = SparkSession.builder.config("spark.jars", "/home/jovyan/work/postgresql-42.2.5.jar") \
#     .master("local").appName("PySpark_Postgres_test").getOrCreate()

In [2]:
# df = spark.read.format("jdbc") \
#     .option("url", "jdbc:postgresql://172.28.0.2:5432/user-name") \
#     .option("driver", "org.postgresql.Driver") \
#     .option("dbtable", "links") \
#     .option("user", "user-name") \
#     .option("password", "strong-password") \
#     .load()

In [3]:
# df.printSchema()

In [4]:
# df.show()

In [5]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark_xls = SparkSession.builder.config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .master("local").appName("excels").getOrCreate()

In [6]:
df_xls = spark_xls.read.format("com.crealytics.spark.excel") \
.option("header", "true") \
.option("inferSchema", "true") \
.option("dataAddress", "'Loan Level'!A1") \
.load("/home/jovyan/work/DE_Rand_test (2).xlsx")

In [7]:
df_xls.show(5)

+--------------------+-----------+--------------------+---------+---------+-----------+-------+-------------+---------+
|             loan_id|loan_amount|         borrower_id|   status|  partner|current_dpd|max_dpd|interest_rate|loan_term|
+--------------------+-----------+--------------------+---------+---------+-----------+-------+-------------+---------+
|1cd246a8-1d42-4e4...|     7.73E7|f06f889b-bf6c-4c0...|     LIVE|FINTECH_A|        0.0|    0.0|         0.12|     21.0|
|5a2a5ac3-0136-462...|     4.61E7|cbb43196-a92f-499...|     LIVE|FINTECH_A|       57.0|   57.0|         0.12|     84.0|
|9613a317-2031-496...|     5.75E7|08035917-3b4e-48c...|CANCELLED|   BANK_A|        0.0|    0.0|         0.12|     63.0|
|07ec998e-a413-41a...|     7.38E7|b233e831-a7a6-44d...|     LIVE|FINTECH_B|       52.0|   57.0|          0.2|    112.0|
|384c2f53-6bad-4ee...|     8.15E7|85611e80-e5c1-458...|  PENDING|FINTECH_B|        0.0|    0.0|          0.2|     28.0|
+--------------------+-----------+------

In [8]:
df_xls.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_amount: double (nullable = true)
 |-- borrower_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- partner: string (nullable = true)
 |-- current_dpd: double (nullable = true)
 |-- max_dpd: double (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- loan_term: double (nullable = true)



In [9]:
df_xls.count()

10000

In [10]:
df_xls = df_xls \
    .withColumn("loan_amount", df_xls.loan_amount.cast('int')) \
    .withColumn("current_dpd", df_xls.current_dpd.cast('int')) \
    .withColumn("max_dpd", df_xls.max_dpd.cast('int')) \
    .withColumn("loan_term", df_xls.loan_term.cast('int'))

In [11]:
df_xls.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- borrower_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- partner: string (nullable = true)
 |-- current_dpd: integer (nullable = true)
 |-- max_dpd: integer (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- loan_term: integer (nullable = true)



In [12]:
from pyspark.sql.functions import col

df_xls.collect()[0]

Row(loan_id='1cd246a8-1d42-4e48-ae55-016d9cb36d60', loan_amount=77300000, borrower_id='f06f889b-bf6c-4c03-84a7-8d38dc965e65', status='LIVE', partner='FINTECH_A', current_dpd=0, max_dpd=0, interest_rate=0.12, loan_term=21)

In [13]:
df_xls.select('status').distinct().collect()

[Row(status='CANCELLED'),
 Row(status='LIVE'),
 Row(status='CLOSED'),
 Row(status='PENDING')]

In [14]:
import pyspark.sql.functions as F

#update all values in 'team' column equal to 'A' to now be 'Atlanta'
df_xls = df_xls.withColumn('status', F.when(df_xls.status=='PENDING', 'SUBMITTED').otherwise(df_xls.status))

In [22]:
df_xls.select('status').distinct().collect()

[Row(status='SUBMITTED'),
 Row(status='CANCELLED'),
 Row(status='LIVE'),
 Row(status='CLOSED')]

In [23]:
# spark_pgsql = SparkSession.builder.config("spark.jars", "/home/jovyan/work/postgresql-42.7.1.jar") \
#     .master("local").appName("PySpark_Postgres_test").getOrCreate()

In [24]:
# df_xls \
#     .select("loan_id","loan_amount","borrower_id","status","partner","current_dpd","max_dpd","interest_rate","loan_term") \
#     .write.format("jdbc")\
#     .option("url", "jdbc:postgresql://172.28.0.2:5432/user-name") \
#     .option("driver", "org.postgresql.Driver") \
#     .option("dbtable", "loan") \
#     .option("user", "user-name") \
#     .option("password", "strong-password") \
#     .option("numPartitions", 20)\
#     .save()

In [30]:
df_xls_df = df_xls \
    .select("loan_id","loan_amount","borrower_id","status","partner","current_dpd","max_dpd","interest_rate","loan_term") \
    .toPandas()

In [31]:
df_xls_df.dtypes

loan_id           object
loan_amount        int32
borrower_id       object
status            object
partner           object
current_dpd        int32
max_dpd            int32
interest_rate    float64
loan_term          int32
dtype: object

In [38]:
!pip install psycopg2-binary

from sqlalchemy import create_engine
import psycopg2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 682.2 kB/s eta 0:00:0000:0100:01


In [40]:
# Replace these values with your PostgreSQL connection details
db_connection = {
    'user': 'user-name',
    'password': 'strong-password',
    'host': '172.28.0.2',
    'port': '5432',
    'database': 'user-name',
}

# Replace 'your_table_name' with the name of your PostgreSQL table
table_name = 'loan'

# Create a connection to the PostgreSQL database
engine = create_engine(f"postgresql+psycopg2://{db_connection['user']}:{db_connection['password']}@{db_connection['host']}:{db_connection['port']}/{db_connection['database']}")

# Assuming you have a DataFrame named 'df' with the data you want to load
df_xls_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the database connection
engine.dispose()

In [41]:
df_xls_2 = spark_xls.read.format("com.crealytics.spark.excel") \
.option("header", "true") \
.option("inferSchema", "true") \
.option("dataAddress", "'Partner Level'!A1") \
.load("/home/jovyan/work/DE_Rand_test (2).xlsx")

In [43]:
df_xls_2.printSchema()

root
 |-- partner: string (nullable = true)
 |-- name: string (nullable = true)



In [45]:
df_xls_df_2 = df_xls_2 \
    .select("partner", "name") \
    .toPandas()

In [47]:
# Replace these values with your PostgreSQL connection details
db_connection = {
    'user': 'user-name',
    'password': 'strong-password',
    'host': '172.28.0.2',
    'port': '5432',
    'database': 'user-name',
}

# Replace 'your_table_name' with the name of your PostgreSQL table
table_name = 'partner'

# Create a connection to the PostgreSQL database
engine = create_engine(f"postgresql+psycopg2://{db_connection['user']}:{db_connection['password']}@{db_connection['host']}:{db_connection['port']}/{db_connection['database']}")

# Assuming you have a DataFrame named 'df' with the data you want to load
df_xls_df_2.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the database connection
engine.dispose()

In [48]:
df_xls.show(5)

+--------------------+-----------+--------------------+---------+---------+-----------+-------+-------------+---------+
|             loan_id|loan_amount|         borrower_id|   status|  partner|current_dpd|max_dpd|interest_rate|loan_term|
+--------------------+-----------+--------------------+---------+---------+-----------+-------+-------------+---------+
|1cd246a8-1d42-4e4...|   77300000|f06f889b-bf6c-4c0...|     LIVE|FINTECH_A|          0|      0|         0.12|       21|
|5a2a5ac3-0136-462...|   46100000|cbb43196-a92f-499...|     LIVE|FINTECH_A|         57|     57|         0.12|       84|
|9613a317-2031-496...|   57500000|08035917-3b4e-48c...|CANCELLED|   BANK_A|          0|      0|         0.12|       63|
|07ec998e-a413-41a...|   73800000|b233e831-a7a6-44d...|     LIVE|FINTECH_B|         52|     57|          0.2|      112|
|384c2f53-6bad-4ee...|   81500000|85611e80-e5c1-458...|SUBMITTED|FINTECH_B|          0|      0|          0.2|       28|
+--------------------+-----------+------

In [119]:
df_xls = df_xls.withColumn(
    'dpd_category',
    F.when((F.col("current_dpd") == 0), "dpd_category0")\
    .when((F.col("current_dpd") >= 1) & (F.col('current_dpd') <= 30), "dpd_category130")\
    .when((F.col("current_dpd") >= 31) & (F.col('current_dpd') <= 60), "dpd_category3160")\
    .otherwise("dpd_category61more")
)

df_xls = df_xls.withColumn(
    'kolektibilitas',
    F.when((F.col("current_dpd") == 0), "lancar")\
    .when((F.col("current_dpd") >= 1) & (F.col('current_dpd') <= 90), "dalam_perhatian_khusus")\
    .when((F.col("current_dpd") >= 91) & (F.col('current_dpd') <= 120), "kurang_lancar")\
    .when((F.col("current_dpd") >= 121) & (F.col('current_dpd') <= 180), "diragukan")\
    .otherwise("macet")
)

In [120]:
df_xls.show(1)

+--------------------+-----------+--------------------+------+---------+-----------+-------+-------------+---------+-------------+--------------+
|             loan_id|loan_amount|         borrower_id|status|  partner|current_dpd|max_dpd|interest_rate|loan_term| dpd_category|kolektibilitas|
+--------------------+-----------+--------------------+------+---------+-----------+-------+-------------+---------+-------------+--------------+
|1cd246a8-1d42-4e4...|   77300000|f06f889b-bf6c-4c0...|  LIVE|FINTECH_A|          0|      0|         0.12|       21|dpd_category0|        lancar|
+--------------------+-----------+--------------------+------+---------+-----------+-------+-------------+---------+-------------+--------------+
only showing top 1 row



In [121]:
dpd_bucket_cat0 = df_xls.select("loan_id", "borrower_id", "loan_amount", "dpd_category").filter(df_xls.dpd_category == "dpd_category0")
dpd_bucket_cat1 = df_xls.select("loan_id", "borrower_id", "loan_amount", "dpd_category").filter(df_xls.dpd_category == "dpd_category130")
dpd_bucket_cat2 = df_xls.select("loan_id", "borrower_id", "loan_amount", "dpd_category").filter(df_xls.dpd_category == "dpd_category3160")
dpd_bucket_cat3 = df_xls.select("loan_id", "borrower_id", "loan_amount", "dpd_category").filter(df_xls.dpd_category == "dpd_category61more")

In [122]:
print(dpd_bucket_cat0.count())
print(dpd_bucket_cat1.count())
print(dpd_bucket_cat2.count())
print(dpd_bucket_cat3.count())

7171
638
589
1602


In [123]:
dpd_bucket_cat0.show()

+--------------------+--------------------+-----------+-------------+
|             loan_id|         borrower_id|loan_amount| dpd_category|
+--------------------+--------------------+-----------+-------------+
|1cd246a8-1d42-4e4...|f06f889b-bf6c-4c0...|   77300000|dpd_category0|
|9613a317-2031-496...|08035917-3b4e-48c...|   57500000|dpd_category0|
|384c2f53-6bad-4ee...|85611e80-e5c1-458...|   81500000|dpd_category0|
|85e2b6fd-3469-459...|ab69fff7-8442-47c...|   50000000|dpd_category0|
|ed975b90-8192-463...|1200a453-62ea-48d...|   11700000|dpd_category0|
|b59e9933-f832-4ea...|b8eb2c87-1cc0-4d0...|   90800000|dpd_category0|
|11ca09a7-1362-4c6...|149c403c-98ea-4f2...|    1100000|dpd_category0|
|6aff95e5-42e4-454...|c45135be-380e-4cf...|   65500000|dpd_category0|
|4c0dc051-26aa-45a...|05a8cb1d-57e1-427...|   31300000|dpd_category0|
|801806de-e6cb-4b9...|f2ab849a-61de-4c9...|   88000000|dpd_category0|
|177516d4-e8a5-414...|e59b45d2-6ca5-47c...|   13900000|dpd_category0|
|1865983f-e89c-42a..

In [124]:
loan_per_dpd = df_xls.select("loan_term", "dpd_category")

In [125]:
loan_per_dpd.show()

+---------+------------------+
|loan_term|      dpd_category|
+---------+------------------+
|       21|     dpd_category0|
|       84|  dpd_category3160|
|       63|     dpd_category0|
|      112|  dpd_category3160|
|       28|     dpd_category0|
|       91|     dpd_category0|
|       49|dpd_category61more|
|      105|     dpd_category0|
|       84|dpd_category61more|
|      112|     dpd_category0|
|       42|     dpd_category0|
|       84|     dpd_category0|
|       35|   dpd_category130|
|       56|     dpd_category0|
|       84|     dpd_category0|
|       28|     dpd_category0|
|       56|  dpd_category3160|
|      105|     dpd_category0|
|       49|  dpd_category3160|
|       77|     dpd_category0|
+---------+------------------+
only showing top 20 rows



In [128]:
from pyspark.sql.functions import sum

In [134]:
# membagi jumlah kredit kurang lancar, diragukan, dan macet dengan total kredit yang disalurkan, 
# kemudian dikali 100%

# Rasio NPL= (Total NPL (Kurang Lancar + Diragukan + Macet)/Total Kredit) X 100%
PL = df_xls.filter(df_xls.kolektibilitas == "lancar").select(sum(dpd_bucket_cat0.loan_amount)).first()[0]
NPL1 = df_xls.filter(df_xls.kolektibilitas == "dalam_perhatian_khusus").select(sum(dpd_bucket_cat0.loan_amount)).first()[0]
NPL2 = df_xls.filter(df_xls.kolektibilitas == "kurang_lancar").select(sum(dpd_bucket_cat0.loan_amount)).first()[0]
NPL3 = df_xls.filter(df_xls.kolektibilitas == "diragukan").select(sum(dpd_bucket_cat0.loan_amount)).first()[0]
NPL4 = 0 if (df_xls.filter(df_xls.kolektibilitas == "macet").select(sum(dpd_bucket_cat0.loan_amount)).first()[0]) == None else (df_xls.filter(df_xls.kolektibilitas == "macet").select(sum(dpd_bucket_cat0.loan_amount)).first()[0])



In [136]:
Total_NPL = NPL2 + NPL3 + NPL4
Credit_Total = PL + NPL1 + NPL2 + NPL3 + NPL4
NPL = (Total_NPL / Credit_Total) * 100
print(NPL)

1.9844710485654897


In [100]:
df_xls.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_amount: integer (nullable = true)
 |-- borrower_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- partner: string (nullable = true)
 |-- current_dpd: integer (nullable = true)
 |-- max_dpd: integer (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- loan_term: integer (nullable = true)
 |-- dpd_category: string (nullable = false)



In [106]:
df_xls.agg({'current_dpd': 'max'}).show()

+----------------+
|max(current_dpd)|
+----------------+
|             179|
+----------------+

